In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, silhouette_score
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [2]:
wordsToCut = set(stopwords.words('english'))
wordsToCut

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [3]:
print ('A' in wordsToCut)

False


In [4]:
#data = pd.read_csv('/Users/daviddeng8/affinity/data/outlet_data.csv')
#data['content'][:100]
data = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/final_data.csv')
#data.loc[261, 'Url']
data

,Url,Author,Date,Header,Body,n_links,Source,Bias,Quality
0,https://abcnews.go.com/Politics/abortion-right...,Devin Dwyer,"Wed, 17 Apr 2019 10:14:00 GMT",Abortion rights group asks Supreme Court to st...,Abortion rights advocates have asked the U.S. ...,3.0,ABC,1.67,49.00
1,https://abcnews.go.com/Politics/appeals-court-...,Ali Dukakis,"Tue, 26 Feb 2019 09:05:00 GMT",Appeals court says special counsel Robert Muel...,A federal appeals court rejected the most dire...,2.0,ABC,0.67,51.67
2,https://abcnews.go.com/Politics/attorney-gener...,Luke Barr,"Wed, 17 Apr 2019 14:02:00 GMT",Attorney general orders some asylum seekers to...,As part of the Trump administration's effort t...,6.0,ABC,-2.75,43.50
3,https://abcnews.go.com/Politics/donald-trump-t...,Meridith McGraw,"Tue, 19 Mar 2019 12:44:00 GMT","Donald Trump and 'the Trump of the Tropics,' B...","President Donald Trump and ""the Trump of the T...",10.0,ABC,-4.33,52.67
4,https://abcnews.go.com/Politics/electoral-coll...,Matthew Dowd,"Tue, 19 Mar 2019 21:39:00 GMT",The Electoral College limits the campaign play...,"U.S Senator Elizabeth Warren, who is competing...",5.0,ABC,-10.00,32.00
...,...,...,...,...,...,...,...,...,...
1670,https://www.wnd.com/2019/04/12-french-churches...,WND Staff,"Tue, 16 Apr 2019 11:53:00 GMT",12 French churches attacked before Notre Dame ...,Only hours after the first flames that eventua...,18.0,WorldNetDaily,19.75,19.75
1671,https://www.wnd.com/2019/04/doj-sued-for-detai...,WND Staff,"Tue, 16 Apr 2019 19:16:00 GMT",DOJ sued for details of payments to Christophe...,Washington watchdog Judicial Watch believes th...,2.0,WorldNetDaily,30.50,21.25
1672,https://www.wnd.com/2019/04/fox-news-stars-pul...,WND Staff,"Tue, 16 Apr 2019 14:23:00 GMT",Fox News stars pull plug on history of church ...,"Two Fox News personalities, as Notre Dame Cath...",7.0,WorldNetDaily,29.75,12.00
1673,https://www.wnd.com/2019/04/major-u-s-bank-shu...,WND Staff,"Tue, 16 Apr 2019 16:37:00 GMT",Major U.S. bank shuts down 'alt-right' accounts,Chase Bank is shutting down accounts of people...,3.0,WorldNetDaily,22.00,25.25


In [5]:
def getPOS(words):
    #return nltk.pos_tag(word)
    return [(word, get_wordnet_pos(word)) for word in words]

In [6]:
def getIndividualPOS(word):
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV} 
    tag = nltk.pos_tag(word[0].upper())
    return tag_dict.get(tag)

In [7]:
#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    return tag_dict.get(tag, wordnet.NOUN)

In [8]:
print (getPOS(['hello', 'bye']))

[('hello', 'n'), ('bye', 'n')]


In [9]:
for index, row in data.iterrows():
    
    #removing stopwords and punctuation
    tokenized = nltk.word_tokenize(row['Body'])
    temp = [word for word in tokenized if word.lower() not in wordsToCut and word not in string.punctuation and word.isalpha()]
    
    #row['Body'] = ' '.join(temp)
    
    #row['Body'] = [word for word in nltk.word_tokenize(row['Body']) if word not in wordsToCut]
    #row['Body'] = row['Body'].lower()
    #nopunc = [char for char in row['Body'] if char not in string.punctuation]
    #nopunc.join('')
    
    #lemmatizing, ONLY KEEPING ENTITIES
    lemmatizer = WordNetLemmatizer()
    partsOfSpeech = getPOS(temp)
    
    lemmatized = [lemmatizer.lemmatize(word, pos) for word, pos in partsOfSpeech if pos == 'n']
    final = ' '.join(lemmatized)
    data.at[index, 'Body'] = final
    
    


In [10]:
data

,Url,Author,Date,Header,Body,n_links,Source,Bias,Quality
0,https://abcnews.go.com/Politics/abortion-right...,Devin Dwyer,"Wed, 17 Apr 2019 10:14:00 GMT",Abortion rights group asks Supreme Court to st...,Abortion right advocate Supreme Court strike L...,3.0,ABC,1.67,49.00
1,https://abcnews.go.com/Politics/appeals-court-...,Ali Dukakis,"Tue, 26 Feb 2019 09:05:00 GMT",Appeals court says special counsel Robert Muel...,appeal court challenge counsel Robert Mueller ...,2.0,ABC,0.67,51.67
2,https://abcnews.go.com/Politics/attorney-gener...,Luke Barr,"Wed, 17 Apr 2019 14:02:00 GMT",Attorney general orders some asylum seekers to...,part Trump administration effort migrant borde...,6.0,ABC,-2.75,43.50
3,https://abcnews.go.com/Politics/donald-trump-t...,Meridith McGraw,"Tue, 19 Mar 2019 12:44:00 GMT","Donald Trump and 'the Trump of the Tropics,' B...",President Donald Trump Trump Tropics president...,10.0,ABC,-4.33,52.67
4,https://abcnews.go.com/Politics/electoral-coll...,Matthew Dowd,"Tue, 19 Mar 2019 21:39:00 GMT",The Electoral College limits the campaign play...,Senator Elizabeth Warren nomination president ...,5.0,ABC,-10.00,32.00
...,...,...,...,...,...,...,...,...,...
1670,https://www.wnd.com/2019/04/12-french-churches...,WND Staff,"Tue, 16 Apr 2019 11:53:00 GMT",12 French churches attacked before Notre Dame ...,hour flame would consume Notre Dame roof spire...,18.0,WorldNetDaily,19.75,19.75
1671,https://www.wnd.com/2019/04/doj-sued-for-detai...,WND Staff,"Tue, 16 Apr 2019 19:16:00 GMT",DOJ sued for details of payments to Christophe...,Washington watchdog Watch FBI information orig...,2.0,WorldNetDaily,30.50,21.25
1672,https://www.wnd.com/2019/04/fox-news-stars-pul...,WND Staff,"Tue, 16 Apr 2019 14:23:00 GMT",Fox News stars pull plug on history of church ...,Two Fox News personality Notre Dame Paris engu...,7.0,WorldNetDaily,29.75,12.00
1673,https://www.wnd.com/2019/04/major-u-s-bank-shu...,WND Staff,"Tue, 16 Apr 2019 16:37:00 GMT",Major U.S. bank shuts down 'alt-right' accounts,Chase Bank account people organization view un...,3.0,WorldNetDaily,22.00,25.25


In [16]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size = 0.5)

In [17]:
vectorizer = TfidfVectorizer(use_idf = True, stop_words = 'english')
#matrix = vectorizer.fit_transform(data['Body'])
matrix = vectorizer.fit_transform(train['Body'])
df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
#list comprehension
#x = np.where(df['électricité'] != 0)
df

,aapl,aaron,aarp,ababa,aback,abandon,abandonment,abbey,abbreviation,abby,...,zumba,zurich,zwiebel,água,åslund,édouard,égalité,électricité,𝙹𝚞𝚊𝚗,𝚁𝚘𝚍𝚛𝚒𝚐𝚞𝚎𝚣
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
'''df = df.drop([columnName for (columnName, columnData) in df.iteritems() if columnName.isalpha() == False], axis=1)
matrix = vectorizer.fit_transform(df)
temp = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
temp'''
matrix = vectorizer.fit_transform(data['Body'])
temp = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
temp

,aa,aaa,aaf,aaliyah,aapl,aaron,aarp,aarseth,ababa,aback,...,édouard,égalité,électricité,élysée,émigré,état,île,øystein,𝙹𝚞𝚊𝚗,𝚁𝚘𝚍𝚛𝚒𝚐𝚞𝚎𝚣
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
number_of_clusters = 50
model = KMeans(n_clusters = number_of_clusters)
model.fit(matrix)

# elbow method, silhouette score
# entity recognition: do the same clustering but with only entities (nouns)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [19]:
km_scores= []
km_silhouette = []
vmeasure_score =[]
db_score = []
for i in range(2,50):
    km = KMeans(n_clusters=i, random_state=0).fit(matrix)
    preds = km.predict(matrix)
    
    print("Score for number of cluster(s) {}: {}".format(i,km.score(matrix)))
    km_scores.append(-km.score(matrix))
    
    silhouette = silhouette_score(matrix,preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))
    

Score for number of cluster(s) 2: -799.5346045941839
Silhouette score for number of cluster(s) 2: 0.007869515052423198
Score for number of cluster(s) 3: -790.4660782583998
Silhouette score for number of cluster(s) 3: 0.007535767122196954
Score for number of cluster(s) 4: -786.5923519614713
Silhouette score for number of cluster(s) 4: 0.008839203913181527
Score for number of cluster(s) 5: -780.7645930289345
Silhouette score for number of cluster(s) 5: 0.010971050894623267
Score for number of cluster(s) 6: -777.9422488756032
Silhouette score for number of cluster(s) 6: 0.011275517499811987
Score for number of cluster(s) 7: -773.2725503292794
Silhouette score for number of cluster(s) 7: 0.014538146669683125
Score for number of cluster(s) 8: -768.2870827224546
Silhouette score for number of cluster(s) 8: 0.016023504916522935
Score for number of cluster(s) 9: -763.1080651034039
Silhouette score for number of cluster(s) 9: 0.019599728676635446
Score for number of cluster(s) 10: -758.75656697

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(km_scores)

In [ ]:
plt.figure(figsize = (15, 10))
plt.scatter(x = [i for i in range(2, 50)], y = km_scores)
plt.xlabel('number of clusters', fontsize=20)
plt.ylabel('elbow method score', fontsize=20)

In [ ]:
plt.figure(figsize = (15, 10))
plt.scatter(x = [i for i in range(2, 50)], y = km_silhouette)
plt.xlabel('number of clusters', fontsize = 20)
plt.ylabel('silhouette score', fontsize = 20)

In [ ]:
number_of_clusters = 50
model = KMeans(n_clusters = number_of_clusters)
model.fit(matrix)

In [ ]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

In [ ]:
import pandas as pd
gapminder_url='https://bit.ly/2cLzoxH'
gapminder = pd.read_csv(gapminder_url)
gapminder

In [ ]:
gapminder_ocean = gapminder[(gapminder.year > 2000) & (gapminder.continent == 'Oceania')]
gapminder_ocean

In [ ]:
new = gapminder_ocean.drop([columnName for (columnName, columnData) in gapminder_ocean.iteritems() if columnName == 'pop'], axis = 1)
new

In [ ]:
new['69'] = [0, 0, 0, 0]
new

In [ ]:
new = new.drop([columnName if ])

### OLD DATA

In [ ]:
data_old = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/outlet_data.csv')

vectorizer_old = TfidfVectorizer(use_idf = True, stop_words = 'english')
matrix_old = vectorizer.fit_transform(data_old['content'])
df_old = pd.DataFrame(matrix_old.toarray(), columns = vectorizer.get_feature_names())
df_old

In [ ]:
number_of_clusters_old = 50
model_old = KMeans(n_clusters = number_of_clusters_old)
model_old.fit(matrix_old)

In [ ]:
order_centroids_old = model_old.cluster_centers_.argsort()[:, ::-1]
terms_old = vectorizer_old.get_feature_names()
for i in range(number_of_clusters_old):
    print("Cluster %d:" % i),
    for ind in order_centroids_old[i, :10]:
        print(' %s' % terms_old[ind]),
    print

In [ ]:
km_scores_old= []
km_silhouette_old = []
vmeasure_score_old =[]
db_score_old = []
for i in range(2,50):
    km_old = KMeans(n_clusters=i, random_state=0).fit(matrix_old)
    preds_old = km_old.predict(matrix_old)
    
    print("Score for number of cluster(s) {}: {}".format(i,km_old.score(matrix)))
    km_scores_old.append(-km_old.score(matrix_old))
    
    silhouette_old = silhouette_score_old(matrix_old,preds_old)
    km_silhouette_old.append(silhouette_old)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette_old))
    

### Spacy Part of speech tagging